In [11]:
import yaml
from glob import glob
import matplotlib.pyplot as plt
import numpy as np


def right_candidate(t0, period, true_t0, true_period, verbose=False):
    t0_check = (
        np.abs((t0 - true_t0 + 0.5 * true_period) % true_period - 0.5 * true_period)
        % period
        < 0.01
    )
    period_check = np.abs(period - true_period) < 0.1
    period_check |= np.abs(2 * period - true_period) < 0.1
    same = np.logical_and(t0_check, period_check)
    if verbose:
        if not same:
            if not t0_check:
                output = f"t0 differ: {t0:.2e} {true_t0:.2e}"
            if not period_check:
                output = f"period differ: {period:.2e} {true_period:.2e}"
        else:
            output = "match"
        return same, output
    else:
        return same


def get_result(f, tool):
    d = yaml.safe_load(open(f, "r"))
    return {
        f"{tool}_found_t0": d["found_t0"],
        f"{tool}_found_period": d["found_period"],
        f"{tool}_found": right_candidate(
            d["found_t0"], d["found_period"], d["transit_t0"], d["planet_period"]
        ),
    }, d


In [12]:
nuance_files = [f"../../data/29923559/nuance_search/{i}.yaml" for i in range(20**2)]
tls_files = [f"../../data/29923559/tls_search/{i}.yaml" for i in range(20**2)]

results = []

for i, (nuance_file, tls_file) in enumerate(zip(nuance_files, tls_files)):
    nuance_results, injected = get_result(nuance_file, "nuance")
    tls_results, _ = get_result(tls_file, "tls")
    results.append({**nuance_results, **tls_results, **injected})


In [13]:
import pandas as pd

df = pd.DataFrame(results)
df


,nuance_found_t0,nuance_found_period,nuance_found,tls_found_t0,tls_found_period,tls_found,delta,found_duration,found_period,found_t0,planet_period,planet_radius,star_amplitude,star_mass,star_period,star_radius,tau,transit_depth,transit_duration,transit_t0
0,1.808243,0.100000,False,0.506955,1.400500,False,3.499619,0.016909,0.100000,1.808243,0.200000,1.594845,0.01559,0.188202,0.501623,0.219066,14.832846,0.004455,0.016909,0.109763
1,2.443083,0.715912,True,0.118055,1.570557,False,3.499619,0.025865,0.715912,2.443083,0.715789,1.594845,0.01559,0.188202,0.501623,0.219066,9.697045,0.004455,0.025865,0.298500
2,1.769493,1.231533,True,0.537564,1.231533,True,3.499619,0.030993,1.231533,1.769493,1.231579,1.594845,0.01559,0.188202,0.501623,0.219066,8.092457,0.004455,0.030993,0.536962
3,2.711172,1.747155,True,0.951412,0.873977,False,3.499619,0.034826,1.747155,2.711172,1.747368,1.594845,0.01559,0.188202,0.501623,0.219066,7.201777,0.004455,0.034826,0.962447
4,7.605680,6.044364,False,0.398180,0.630883,False,3.499619,0.010000,6.044364,7.605680,2.263158,1.594845,0.01559,0.188202,0.501623,0.219066,6.606877,0.004455,0.037962,2.188541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,3.857020,7.936794,True,3.856469,7.937884,True,0.466616,0.057676,7.936794,3.857020,7.936842,4.367662,0.01559,0.188202,0.501623,0.219066,4.348625,0.033411,0.057676,3.856661
396,2.604226,8.452415,True,2.603873,8.451325,True,0.466616,0.058899,8.452415,2.604226,8.452632,4.367662,0.01559,0.188202,0.501623,0.219066,4.258309,0.033411,0.058899,2.604053
397,1.904217,8.968037,True,1.907661,8.966947,True,0.466616,0.060074,8.968037,1.904217,8.968421,4.367662,0.01559,0.188202,0.501623,0.219066,4.175058,0.033411,0.060074,1.904180
398,7.032061,9.484748,True,2.296023,4.736234,False,0.466616,0.061204,9.484748,7.032061,9.484211,4.367662,0.01559,0.188202,0.501623,0.219066,4.097957,0.033411,0.061204,7.031015


[{'delta': 1.6473501717337784,
  'found_duration': 0.06120397542444834,
  'found_period': 10.808140814081407,
  'found_t0': 7.61401314689283,
  'planet_period': 9.48421052631579,
  'planet_radius': 2.3245331953285415,
  'star_amplitude': 0.01559,
  'star_mass': 0.188202,
  'star_period': 0.50162256,
  'star_radius': 0.219066,
  'tau': 4.097957334644177,
  'transit_depth': 0.009463683112129142,
  'transit_duration': 0.06120397542444834,
  'transit_t0': 8.940305949418253,
  'found': False},
 {'delta': 2.500590253853881,
  'found_duration': 0.016909180836310725,
  'found_period': 0.39977997799779974,
  'found_t0': 2.0862600430358635,
  'planet_period': 0.2,
  'planet_radius': 1.8867199781783475,
  'star_amplitude': 0.01559,
  'star_mass': 0.188202,
  'star_period': 0.50162256,
  'star_radius': 0.219066,
  'tau': 14.832846276113424,
  'transit_depth': 0.0062345280183240225,
  'transit_duration': 0.016909180836310725,
  'transit_t0': 0.08153740561605723,
  'found': False},
 {'delta': 0.8691